## Transformer

In [11]:
# Author: Haozhou Xu
# PID: A69032157
# Date: 2024.10.25

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

class Attention(nn.Module):
    def __init__(self, embed_size, head_num):
        super(Attention, self).__init__()
        assert embed_size % head_num == 0
        self.embed_size = embed_size
        self.head_num = head_num
        self.head_dim = embed_size // head_num

        # Separate Q, K, V layers for each attention head
        self.q = nn.Linear(embed_size, embed_size, bias=False)
        self.k = nn.Linear(embed_size, embed_size, bias=False)
        self.v = nn.Linear(embed_size, embed_size, bias=False)
        self.fc = nn.Linear(embed_size, embed_size)

    def forward(self, x, mask=None):
        N, seq_len, num_features = x.shape

        # Split the embedding into heads
        q = self.q(x).reshape(N, seq_len, self.head_num, self.head_dim).permute(0, 2, 1, 3)
        k = self.k(x).reshape(N, seq_len, self.head_num, self.head_dim).permute(0, 2, 1, 3)
        v = self.v(x).reshape(N, seq_len, self.head_num, self.head_dim).permute(0, 2, 1, 3)

        # Calculate
        score = torch.matmul(q, k.permute(0, 1, 3, 2)) / np.sqrt(self.head_dim)
        if mask is not None:
            mask = mask.unsqueeze(1)  # (batch_size, 1, seq_len, seq_len)
            score = score.masked_fill(mask == 0, float('-inf'))
        weight = F.softmax(score, dim=-1)
        # Perhaps we can add dropout here
        # weight = self.dropout(weight)
        out = torch.matmul(weight, v).permute(0, 2, 1, 3).reshape(N, seq_len, self.embed_size)
        out = self.fc(out)
        return out


class FNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0):
        super(FNN, self).__init__()
        assert num_layers >= 1
        self.layers = nn.ModuleList(
            [nn.Sequential(
                nn.Linear(hidden_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout)
            ) for i in range(num_layers - 1)])
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        for layer in self.layers:
            x = layer(x)
        x = self.fc2(x)
        return x


class TransformerEncoder(nn.Module):
    def __init__(self, embed_size, head_num, hidden_size, num_layers=1, dropout=0):
        super(TransformerEncoder, self).__init__()
        assert num_layers >= 1
        self.layers = nn.ModuleList(
            [nn.Sequential(
                Attention(embed_size, head_num),
                FNN(embed_size, hidden_size, embed_size, num_layers=1, dropout=dropout)
            ) for i in range(num_layers)])
        self.norm = nn.LayerNorm(embed_size)

    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer[0](x, mask) + x
            x = layer[1](x) + x
        return self.norm(x)
    
    def encode(self, x, mask=None):
        x = self.forward(x, mask)
        x = x.mean(dim=1)
        return x


In [15]:
# 测试TransformerEncoder类
embed_size = 32
head_num = 4
hidden_size = 64
num_layers = 2
seq_len = 10
batch_size = 3

# 模拟输入数据和mask
input_data = torch.randn(batch_size, seq_len, embed_size)
print("输入数据形状:", input_data.shape)  # 期望形状: (batch_size, seq_len, embed_size)
mask = torch.ones(batch_size, seq_len, seq_len)  # 可选的mask

# 初始化模型并测试
model = TransformerEncoder(embed_size, head_num, hidden_size, num_layers=num_layers, dropout=0.1)
output = model(input_data, mask)
print("TransformerEncoder 输出形状:", output.shape)  # 期望形状: (batch_size, seq_len, embed_size)
encoded_output = model.encode(input_data, mask)
print("Encoded 输出形状:", encoded_output.shape)  # 期望形状: (batch_size, embed_size)


输入数据形状: torch.Size([3, 10, 32])
TransformerEncoder 输出形状: torch.Size([3, 10, 32])
Encoded 输出形状: torch.Size([3, 32])


## 调库

In [17]:
import torch
import torch.nn as nn

class FNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_heads, num_layers, dropout=0.1):
        super().__init__()
        
        # Token Embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Positional Encoding Layer
        self.pos_encoder = nn.Parameter(torch.zeros(1, 32, embed_dim))  # 32 is block_size from main.py
        
        # Transformer Encoder Layer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=4*embed_dim,  # 标准transformer使用4倍的embed_dim
            dropout=dropout,
            batch_first=True
        )
        
        # Transformer Encoder
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )
        
        # Classifier layers
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        # Generate attention mask (1 for non-padding, 0 for padding)
        mask = (x != 0).float()
        
        # Embedding layer
        x = self.embedding(x)
        
        # Add positional encoding
        x = x + self.pos_encoder[:, :x.size(1), :]
        
        # Create attention mask for transformer
        # Convert boolean mask to float and then to bool
        attention_mask = (mask == 0).bool()
        
        # Transformer layers
        x = self.transformer(x, src_key_padding_mask=attention_mask)
        
        # Mean pooling over sequence length
        x = (x * mask.unsqueeze(-1)).sum(dim=1) / mask.sum(dim=1, keepdim=True)
        
        # Classification layers
        x = self.classifier(x)
        
        return x

In [22]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import os

from tokenizer import SimpleTokenizer
from dataset import SpeechesClassificationDataset, LanguageModelingDataset

seed = 42
PYTORCH_ENABLE_MPS_FALLBACK=1
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

""" Hyperparameters to use for training to roughly match 
the numbers mentioned in the assignment description """
batch_size = 16  # Number of independent sequences  we will process in parallel
block_size = 32  # Maximum context length for predictions
learning_rate = 1e-3  # Learning rate for the optimizer
n_embd = 64  # Embedding dimension
n_head = 2  # Number of attention heads
n_layer = 4  # Number of transformer layers


eval_interval = 100  # How often to evaluate train and test perplexity during training
max_iters = 500 # For language modeling, we can process all the batches for the entire dataset, but that takes a while, so we'll limit it to 500 iterations. For batch size of 16 and block size of  32, this is roughly, this is  500 * 16 * 32 = 256000 tokens, SOTA LMs are trained on trillions of tokens, so this is a very small dataset.
eval_iters = 200  # Number of iterations to evaluate perplexity on the test set


## classifier training hyperparameters. It is a simple 1 hidden layer feedforward network, with input 
## size of 64, hidden size of 50 and output size of 3.

n_input = 64  # Input size for the classifier, should match the embedding size of the transformer
n_hidden = 100  # Hidden size for the classifier
n_output = 3  # Output size for the classifier, we have 3 classes
epochs_CLS = 15 # epochs for classifier training

def load_texts(directory):
    """
    This function loads all texts from the specified directory, ignoring any files with "test" in their name. The text is used for "training" the tokenizer. Since our tokenizer is simple, we don't need to do any training, but we still need to ignore the test data. 
    """

    texts = []
    files = os.listdir(directory)
    for filename in files: 
        if "test" in filename:  ## don't "read test files"
            continue
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            texts.append(file.read())
    return texts



def collate_batch(batch):
    """ Collate a batch of data into a single tensor with padding."""
    data, labels = zip(*batch)  # Separate the data and labels
    # Pad sequences to the fixed length
    padded_sequences = pad_sequence(data, batch_first=True, padding_value=0)
    padded_sequences = padded_sequences[:, :block_size]  # Truncate if longer
    # Add padding if shorter
    padded_sequences = torch.nn.functional.pad(padded_sequences, (0, max(0, block_size - padded_sequences.shape[1])), "constant", 0)
    labels = torch.stack(labels)  
    return padded_sequences, labels

def compute_classifier_accuracy(classifier, data_loader):
    """ Compute the accuracy of the classifier on the data in data_loader."""
    classifier.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for X, Y in data_loader:
            X, Y = X.to(device), Y.to(device)
            outputs = classifier(X)
            _, predicted = torch.max(outputs.data, 1)
            total_correct += (predicted == Y).sum().item()
            total_samples += Y.size(0)
        accuracy = (100 * total_correct / total_samples)
        classifier.train()
        return accuracy


def compute_perplexity(decoderLMmodel, data_loader, eval_iters=100):
    """ Compute the perplexity of the decoderLMmodel on the data in data_loader.
    Make sure to use the cross entropy loss for the decoderLMmodel.
    """
    decoderLMmodel.eval()
    losses= []
    for X, Y in data_loader:
        X, Y = X.to(device), Y.to(device)
        loss = decoderLMmodel(X, Y) # your model should be computing the cross entropy loss
        losses.append(loss.item())
        total_loss += loss.item()
        if len(losses) >= eval_iters: break


    losses = torch.tensor(losses)
    mean_loss = losses.mean()
    perplexity = torch.exp(mean_loss).item()  # Calculate perplexity as exp(mean loss)

    decoderLMmodel.train()
    return perplexity

def train_CLS_model(tokenizer, train_CLS_loader, epochs_CLS):
    cls = FNNClassifier(tokenizer.vocab_size, n_embd, n_hidden, n_output, n_head, n_layer, dropout=0.1).to(device)
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(cls.parameters(), lr=0.001)
    cls.to(device)
    
    for epoch in range(epochs_CLS):
        epoch_loss = 0
        total_correct = 0
        total_samples = 0
        
        for xb, yb in train_CLS_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            outputs = cls(xb)
            loss = loss_fn(outputs, yb)
            loss.backward()
            optimizer.step()
            
            # Calculate the total loss and accuracy
            epoch_loss += loss.item()
            total_correct += (outputs.argmax(1) == yb).sum().item()
            total_samples += yb.size(0)
        
        # Sum of the loss for the entire epoch
        avg_loss = epoch_loss / len(train_CLS_loader)
        accuracy = (total_correct / total_samples) * 100
        print(f"Epoch {epoch + 1}/{epochs_CLS}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    
    return cls

print("Loading data and creating tokenizer ...")
texts = load_texts('speechesdataset')
tokenizer = SimpleTokenizer(' '.join(texts)) # create a tokenizer from the data
print("Vocabulary size is", tokenizer.vocab_size)

train_CLS_dataset = SpeechesClassificationDataset(tokenizer, "speechesdataset/train_CLS.tsv")
train_CLS_loader = DataLoader(train_CLS_dataset, batch_size=batch_size,collate_fn=collate_batch,shuffle=True)


inputfile = "speechesdataset/train_LM.txt"
with open(inputfile, 'r', encoding='utf-8') as f:
    lmtrainText = f.read()
train_LM_dataset = LanguageModelingDataset(tokenizer, lmtrainText,  block_size)
train_LM_loader = DataLoader(train_LM_dataset, batch_size=batch_size, shuffle=True)

# for the classification task, you will train for a fixed number of epochs like this:
# CLS training code here
cls = train_CLS_model(tokenizer, train_CLS_loader, epochs_CLS)
test_CLS_dataset = SpeechesClassificationDataset(tokenizer, "speechesdataset/test_CLS.tsv")
test_CLS_loader = DataLoader(test_CLS_dataset, batch_size=batch_size, collate_fn=collate_batch)
compute_classifier_accuracy(cls, test_CLS_loader)
# for the language modeling task, you will iterate over the training data for a fixed number of iterations like this:
for i, (xb, yb) in enumerate(train_LM_loader):
    if i >= max_iters:
        break
    xb, yb = xb.to(device), yb.to(device)
    # LM training code here

Loading data and creating tokenizer ...
Vocabulary size is 5755
Epoch 1/15, Loss: 1.0939, Accuracy: 37.91%
Epoch 2/15, Loss: 1.0809, Accuracy: 43.31%
Epoch 3/15, Loss: 1.0757, Accuracy: 44.55%
Epoch 4/15, Loss: 1.0736, Accuracy: 44.79%
Epoch 5/15, Loss: 1.0714, Accuracy: 44.55%
Epoch 6/15, Loss: 1.0682, Accuracy: 44.74%
Epoch 7/15, Loss: 1.0702, Accuracy: 44.65%
Epoch 8/15, Loss: 1.0690, Accuracy: 44.55%
Epoch 9/15, Loss: 1.0682, Accuracy: 44.60%
Epoch 10/15, Loss: 1.0691, Accuracy: 44.74%
Epoch 11/15, Loss: 1.0667, Accuracy: 44.74%
Epoch 12/15, Loss: 1.0654, Accuracy: 44.79%
Epoch 13/15, Loss: 1.0663, Accuracy: 44.84%
Epoch 14/15, Loss: 1.0635, Accuracy: 44.84%
Epoch 15/15, Loss: 1.0632, Accuracy: 44.74%


NotImplementedError: The operator 'aten::_nested_tensor_from_mask_left_aligned' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [29]:
class FNNClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, num_heads, num_layers, dropout=0.1):
        super().__init__()
        
        # Token Embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Positional Encoding Layer
        self.pos_encoder = nn.Parameter(torch.zeros(1, 32, embed_dim))
        
        # Transformer Encoder Layer
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim,  # 使用指定的hidden_dim
            dropout=dropout,
            batch_first=True,
            activation='gelu'  # 使用GELU激活函数
        )
        
        # Transformer Encoder
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
            norm=nn.LayerNorm(embed_dim)
        )
        
        # Classifier layers - 使用更深的分类器
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.LayerNorm(hidden_dim // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        
        # 初始化权重
        self._init_weights()
    
    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
    
    def forward(self, x):
        # Generate attention mask (1 for non-padding, 0 for padding)
        mask = (x != 0).float()
        
        # Embedding layer
        x = self.embedding(x)
        
        # Add positional encoding
        x = x + self.pos_encoder[:, :x.size(1), :]
        
        # Create attention mask for transformer
        attention_mask = (mask == 0).bool()
        
        # Transformer layers
        x = self.transformer(x, src_key_padding_mask=attention_mask)
        
        # 使用 [CLS] token 或序列平均值
        # 这里我们使用加权平均，给予较短序列更高的权重
        weights = mask / (mask.sum(dim=1, keepdim=True) + 1e-8)
        x = (x * weights.unsqueeze(-1)).sum(dim=1)
        
        # Classification layers
        x = self.classifier(x)
        
        return x

In [30]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import os

from tokenizer import SimpleTokenizer
from dataset import SpeechesClassificationDataset, LanguageModelingDataset

seed = 42
PYTORCH_ENABLE_MPS_FALLBACK=1
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

""" Hyperparameters to use for training to roughly match 
the numbers mentioned in the assignment description """
batch_size = 16  # Number of independent sequences  we will process in parallel
block_size = 32  # Maximum context length for predictions
learning_rate = 1e-3  # Learning rate for the optimizer
n_embd = 64  # Embedding dimension
n_head = 2  # Number of attention heads
n_layer = 4  # Number of transformer layers


eval_interval = 100  # How often to evaluate train and test perplexity during training
max_iters = 500 # For language modeling, we can process all the batches for the entire dataset, but that takes a while, so we'll limit it to 500 iterations. For batch size of 16 and block size of  32, this is roughly, this is  500 * 16 * 32 = 256000 tokens, SOTA LMs are trained on trillions of tokens, so this is a very small dataset.
eval_iters = 200  # Number of iterations to evaluate perplexity on the test set


## classifier training hyperparameters. It is a simple 1 hidden layer feedforward network, with input 
## size of 64, hidden size of 50 and output size of 3.

n_input = 64  # Input size for the classifier, should match the embedding size of the transformer
n_hidden = 100  # Hidden size for the classifier
n_output = 3  # Output size for the classifier, we have 3 classes
epochs_CLS = 15 # epochs for classifier training

def load_texts(directory):
    """
    This function loads all texts from the specified directory, ignoring any files with "test" in their name. The text is used for "training" the tokenizer. Since our tokenizer is simple, we don't need to do any training, but we still need to ignore the test data. 
    """

    texts = []
    files = os.listdir(directory)
    for filename in files: 
        if "test" in filename:  ## don't "read test files"
            continue
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            texts.append(file.read())
    return texts



def collate_batch(batch):
    """ Collate a batch of data into a single tensor with padding."""
    data, labels = zip(*batch)  # Separate the data and labels
    # Pad sequences to the fixed length
    padded_sequences = pad_sequence(data, batch_first=True, padding_value=0)
    padded_sequences = padded_sequences[:, :block_size]  # Truncate if longer
    # Add padding if shorter
    padded_sequences = torch.nn.functional.pad(padded_sequences, (0, max(0, block_size - padded_sequences.shape[1])), "constant", 0)
    labels = torch.stack(labels)  
    return padded_sequences, labels

def compute_classifier_accuracy(classifier, data_loader):
    """ Compute the accuracy of the classifier on the data in data_loader."""
    classifier.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for X, Y in data_loader:
            X, Y = X.to(device), Y.to(device)
            outputs = classifier(X)
            _, predicted = torch.max(outputs.data, 1)
            total_correct += (predicted == Y).sum().item()
            total_samples += Y.size(0)
        accuracy = (100 * total_correct / total_samples)
        classifier.train()
        return accuracy


def compute_perplexity(decoderLMmodel, data_loader, eval_iters=100):
    """ Compute the perplexity of the decoderLMmodel on the data in data_loader.
    Make sure to use the cross entropy loss for the decoderLMmodel.
    """
    decoderLMmodel.eval()
    losses= []
    for X, Y in data_loader:
        X, Y = X.to(device), Y.to(device)
        loss = decoderLMmodel(X, Y) # your model should be computing the cross entropy loss
        losses.append(loss.item())
        total_loss += loss.item()
        if len(losses) >= eval_iters: break


    losses = torch.tensor(losses)
    mean_loss = losses.mean()
    perplexity = torch.exp(mean_loss).item()  # Calculate perplexity as exp(mean loss)

    decoderLMmodel.train()
    return perplexity

def train_CLS_model(tokenizer, train_CLS_loader, epochs_CLS):
    cls = FNNClassifier(tokenizer.vocab_size, n_embd, n_hidden, n_output, n_head, n_layer, dropout=0.1).to(device)
    loss_fn = torch.nn.CrossEntropyLoss()
    
    # 使用Adam优化器，并添加学习率调度器
    optimizer = torch.optim.Adam(cls.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', 
                                                         factor=0.5, patience=2, 
                                                         verbose=True)
    
    cls.to(device)
    best_accuracy = 0
    
    for epoch in range(epochs_CLS):
        cls.train()
        epoch_loss = 0
        total_correct = 0
        total_samples = 0
        
        for xb, yb in train_CLS_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            
            outputs = cls(xb)
            loss = loss_fn(outputs, yb)
            
            # 添加L2正则化
            l2_lambda = 0.001
            l2_reg = torch.tensor(0.).to(device)
            for param in cls.parameters():
                l2_reg += torch.norm(param)
            loss += l2_lambda * l2_reg
            
            loss.backward()
            
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(cls.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            epoch_loss += loss.item()
            total_correct += (outputs.argmax(1) == yb).sum().item()
            total_samples += yb.size(0)
        
        avg_loss = epoch_loss / len(train_CLS_loader)
        accuracy = (total_correct / total_samples) * 100
        
        print(f"Epoch {epoch + 1}/{epochs_CLS}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
        
        # 更新学习率
        scheduler.step(accuracy)
        
        # 保存最佳模型
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(cls.state_dict(), 'best_model.pth')
    
    # 加载最佳模型
    cls.load_state_dict(torch.load('best_model.pth'))
    return cls

print("Loading data and creating tokenizer ...")
texts = load_texts('speechesdataset')
tokenizer = SimpleTokenizer(' '.join(texts)) # create a tokenizer from the data
print("Vocabulary size is", tokenizer.vocab_size)

train_CLS_dataset = SpeechesClassificationDataset(tokenizer, "speechesdataset/train_CLS.tsv")
train_CLS_loader = DataLoader(train_CLS_dataset, batch_size=batch_size,collate_fn=collate_batch,shuffle=True)


inputfile = "speechesdataset/train_LM.txt"
with open(inputfile, 'r', encoding='utf-8') as f:
    lmtrainText = f.read()
train_LM_dataset = LanguageModelingDataset(tokenizer, lmtrainText,  block_size)
train_LM_loader = DataLoader(train_LM_dataset, batch_size=batch_size, shuffle=True)

# for the classification task, you will train for a fixed number of epochs like this:
# CLS training code here
cls = train_CLS_model(tokenizer, train_CLS_loader, epochs_CLS)
test_CLS_dataset = SpeechesClassificationDataset(tokenizer, "speechesdataset/test_CLS.tsv")
test_CLS_loader = DataLoader(test_CLS_dataset, batch_size=batch_size, collate_fn=collate_batch)
test_acc = compute_classifier_accuracy(cls, test_CLS_loader)
print("Accuracy on test set:", test_acc)
# for the language modeling task, you will iterate over the training data for a fixed number of iterations like this:
for i, (xb, yb) in enumerate(train_LM_loader):
    if i >= max_iters:
        break
    xb, yb = xb.to(device), yb.to(device)
    # LM training code here

Loading data and creating tokenizer ...
Vocabulary size is 5755
Epoch 1/15, Loss: 1.3763, Accuracy: 42.40%
Epoch 2/15, Loss: 1.1286, Accuracy: 60.28%
Epoch 3/15, Loss: 0.9233, Accuracy: 74.19%
Epoch 4/15, Loss: 0.7257, Accuracy: 85.23%
Epoch 5/15, Loss: 0.5805, Accuracy: 90.06%
Epoch 6/15, Loss: 0.4900, Accuracy: 93.98%
Epoch 7/15, Loss: 0.4259, Accuracy: 95.70%
Epoch 8/15, Loss: 0.4139, Accuracy: 96.03%
Epoch 9/15, Loss: 0.4110, Accuracy: 95.70%
Epoch 10/15, Loss: 0.3757, Accuracy: 96.80%
Epoch 11/15, Loss: 0.3594, Accuracy: 97.04%
Epoch 12/15, Loss: 0.3306, Accuracy: 97.32%
Epoch 13/15, Loss: 0.3248, Accuracy: 97.42%
Epoch 14/15, Loss: 0.3231, Accuracy: 97.42%
Epoch 15/15, Loss: 0.2900, Accuracy: 98.04%


NotImplementedError: The operator 'aten::_nested_tensor_from_mask_left_aligned' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.